# EXAMPLE DATA

In [1]:
# means
mn1 = 0.08
mn2 = 0.12
mn3 = 0.15

# std devs
sd1 = 0.15
sd2 = 0.25
sd3 = 0.35

# correlations
c12 = 0.15
c13 = 0.60
c23 = 0.30

# savings rate
rs = 0.02

# borrowing rate (should be no less thansavings rate)
rb = 0.05

# risk aversion
raver = 6

# INSTALL LIBRARIES

In [2]:
!pip install cvxopt


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# CREATE ARRAYS

In [3]:
import numpy as np 

mn = np.array([mn1, mn2, mn3])
sd = np.array([sd1, sd2, sd3])

corr = np.identity(3)
corr[0, 1] = corr[1, 0] = c12
corr[0, 2] = corr[2, 0] = c13
corr[1, 2] = corr[2, 1] = c23

cov = np.diag(sd) @ corr @ np.diag(sd)

# CHECK CORRELATIONS

In [4]:
if np.all(np.linalg.eigvals(cov) > 0):
    print("correlations are acceptable")
else:
    print("correlations are inconsistent")

correlations are acceptable


# CALCULATE FRONTIER OF RISKY ASSETS

In [5]:
# global minimum variance portfolio
w = np.linalg.solve(cov, np.ones(3))
gmv = w / np.sum(w)
gmv_mn = gmv @ mn
gmv_sd = np.sqrt(gmv @ cov @ gmv)

# second frontier portfolio
w = np.linalg.solve(cov, mn)
second_port = w / np.sum(w)
second_mn = second_port @ mn 
second_sd = np.sqrt(second_port @ cov @ second_port)

# means to display
min_mn = 0
max_mn = 1.2*np.max(mn)
mns = np.linspace(min_mn, max_mn, 101)

# portfolio weights and risks
gmv_wt = (mns - second_mn) / (gmv_mn - second_mn)
second_wt = 1 - gmv_wt
ports = (
    gmv_wt.reshape(-1, 1) * gmv.reshape(1, -1) + 
    second_wt.reshape(-1, 1) * second_port.reshape(1, -1)
)
vr = np.diag(ports @ cov @ ports.T)
sds = np.sqrt(vr)


# CALCULATE OPTIMAL PORTFOLIO

Solve for (xs, xb, w1, w2, w3) where xs = amount saved, xb = amount borrowed, and w1, w2, w3 are weights on risky assets.  Portfolio return is

    xs*rs - xb*rb + w1*r1 + w2*r2 + w3*r3

Must have xs>=0, xb>=0 (can't borrow at saving rate or save at borrowing rate) and 

    w1 + w2 + w3 + xs = 1 + xb

In [6]:
from cvxopt import matrix
from cvxopt.solvers import qp

# to define variance penalty
P = np.zeros((5, 5))
P[2:,2:] = raver*cov

# to define (minus) expected return 
q = [-rs, rb, -mn1, -mn2, -mn3]

# to impose -xs <= 0 and -xb <= 0
G = np.zeros((2, 5))
G[0, 0] = G[1, 1] = -1
h = np.zeros(2)

# to impose xs - xb + w1 + w2 + w3 = 1
A = [1., -1., 1., 1., 1.]
b = [1.]

# create cvxopt matrix objects
P = matrix(P, (5, 5))
q = matrix(q, (5, 1))
G = matrix(G, (2, 5))
h = matrix(h, (2, 1))
A = matrix(A, (1, 5))
b = matrix(b, (1, 1))

# compute optimum by quadratic programming
solution = qp(P=P, q=q, G=G, h=h, A=A, b=b)
xs, xb, w1, w2, w3 = solution["x"]
optimum = np.array([w1, w2, w3])

# expected return and risk
optimum_mn = xs*rs - xb*rb + optimum @ mn
optimum_sd = np.sqrt(optimum @ cov @ optimum)

     pcost       dcost       gap    pres   dres
 0: -4.0439e-02 -4.5721e-03  2e+00  2e+00  1e+00
 1: -1.5522e-02 -9.4543e-02  8e-02  4e-02  4e-02
 2: -3.2415e-02 -4.4584e-02  1e-02  1e-15  2e-17
 3: -4.3552e-02 -4.3904e-02  4e-04  4e-16  6e-18
 4: -4.3665e-02 -4.3668e-02  4e-06  2e-16  1e-17
 5: -4.3666e-02 -4.3666e-02  4e-08  2e-16  2e-17
Optimal solution found.


# CALCULATE TANGENCY PORTFOLIOS

In [7]:
if gmv_mn < rs:
    print(
        """ 
        Figure may be incorrect.  For these parameter values, all
        efficient portfolios involve shorting risky assets and investing
        more than 100% at the savings rate.
        """ 
    )
if gmv_mn < rb:
    print(
        """
        Figure may be incorrect.  For these parameter values, it is never
        efficient to borrow at the borrowing rate.
        """
    )

In [8]:
# tangency at savings rate
tangs = np.linalg.solve(cov, mn-rs)
tangs = tangs / np.sum(tangs)
tangs_mn = tangs @ mn
tangs_sd = np.sqrt(tangs @ cov @ tangs)

# tangency at borrowing rate
tangb = np.linalg.solve(cov, mn-rb)
tangb = tangb / np.sum(tangb)
tangb_mn = tangb @ mn
tangb_sd = np.sqrt(tangb @ cov @ tangb)

# FIGURE 1

In [9]:
import plotly.graph_objects as go 

# frontier
string = 'asset 1: %{customdata[0]:.0%}<br>'
string += 'asset 2: %{customdata[1]:.0%}<br>'
string += 'asset 3: %{customdata[2]:.0%}<br>'
string += '<extra></extra>'
tr1 = go.Scatter(
    x=sds,
    y=mns,
    mode="lines",
    customdata=ports,
    hovertemplate=string,
    name="frontier with risky assets only"
)

# three assets
tr2 = go.Scatter( 
    x=sd,
    y=mn,
    text=["Asset 1", "Asset 2", "Asset 3"],
    hovertemplate="%{text}<extra></extra>",
    mode="markers",
    marker=dict(size=15),
    showlegend=False
)

# two tangency portfolios
string = 'asset 1: %{customdata[0]:.0%}<br>'
string += 'asset 2: %{customdata[1]:.0%}<br>'
string += 'asset 3: %{customdata[2]:.0%}<br>'
string += '<extra></extra>'
tr3 = go.Scatter(
    x=[tangs_sd, tangb_sd],
    y=[tangs_mn, tangb_mn],
    customdata=np.array([tangs, tangb]),
    hovertemplate=string,
    mode="markers",
    marker=dict(size=15),
    showlegend=False
)

# efficient portfolios with saving
ports1 = np.arange(0, 1.01, 0.01).reshape(-1, 1) * tangs.reshape(1, -1)
x1 = np.sqrt(np.diag(ports1 @ cov @ ports1.T))
y1 = rs + ports1 @ (mn-rs)
tr4 = go.Scatter(
    x=x1,
    y=y1,
    mode="lines",
    customdata=ports1,
    hovertemplate=string,
    name = "saving"

)

# efficient portfolios with borrowing
ports2 = np.arange(1, 2.01, 0.01).reshape(-1, 1) * tangb.reshape(1, -1)
x2 = np.sqrt(np.diag(ports2 @ cov @ ports2.T))
y2 = rb + ports2 @ (mn-rb)
tr5 = go.Scatter(
    x=x2,
    y=y2,
    mode="lines",
    customdata=ports2,
    hovertemplate=string,
    name="borrowing"
)

# optimal portfolio
tr6= go.Scatter(
    x=[optimum_sd],
    y=[optimum_mn],
    customdata=optimum.reshape(1, -1),
    hovertemplate=string,
    mode="markers",
    marker=dict(size=18, symbol="star"),
    name="optimum",
    legendrank=1
)

# indifference curve
utility = optimum_mn - 0.5 * raver * optimum_sd ** 2
x = np.linspace(0, 2*optimum_sd, 51)
y = utility + 0.5 * raver * x ** 2
string = "equally as good as<br>expected return = %{text:.1%}<br>standard deviation = %{customdata:.1%}<extra></extra>"
tr7 = go.Scatter(
    x=x,
    y=y,
    text=[optimum_mn]*len(x),
    customdata=[optimum_sd]*len(x),
    mode="lines",
    line=dict(dash="dot"),
    hovertemplate=string,
    name="indifference curve",
    legendrank=2,
    )

fig = go.Figure()
for trace in (tr1, tr2, tr3, tr4, tr5, tr6, tr7):
    fig.add_trace(trace)

fig.update_layout(
    xaxis_title="Standard Deviation",
    yaxis_title="Expected Return",
    xaxis_rangemode="tozero",
    yaxis_rangemode="tozero",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
    legend=dict(
        xanchor="left",
        yanchor="top",
        y=0.99,
        x=0.01,
    )
) 
fig.show()

# FIGURE 2

In [12]:
from cvxopt.solvers import options
options["show_progress"] = False

# optimal allocation to risky assets depending on risk aversion
def cal(ra):
    # to define variance penalty
    P = np.zeros((5, 5))
    P[2:,2:] = ra*cov

    # to define (minus) expected return 
    q = [-rs, rb, -mn1, -mn2, -mn3]

    # to impose -xs <= 0 and -xb <= 0
    G = np.zeros((2, 5))
    G[0, 0] = G[1, 1] = -1
    h = np.zeros(2)

    # to impose xs - xb + w1 + w2 + w3 = 1
    A = [1., -1., 1., 1., 1.]
    b = [1.]

    # create cvxopt matrix objects
    P = matrix(P, (5, 5))
    q = matrix(q, (5, 1))
    G = matrix(G, (2, 5))
    h = matrix(h, (2, 1))
    A = matrix(A, (1, 5))
    b = matrix(b, (1, 1))

    # compute optimum by quadratic programming
    solution = qp(P=P, q=q, G=G, h=h, A=A, b=b)
    xs, xb, w1, w2, w3 = solution["x"]
    return w1 + w2 + w3

ravers = np.linspace(0.5, 20, 196)
allocs = [cal(ra) for ra in ravers]
string = "risk aversion = %{x:.1f}<br>allocation to risky assets = %{y:.1%}<extra></extra>"
trace = go.Scatter(
    x=ravers,
    y=allocs,
    mode="lines",
    hovertemplate=string
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Risk Aversion",
    yaxis_title="Optimal Allocation to Risky Assets",
    yaxis_tickformat=".0%",
    template="plotly_white",
    showlegend=False
) 
fig.show()